In [74]:
import pandas as pd

In [75]:
df = pd.read_csv('커피빈매장.csv')
df

,Name,Address,Tel
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330
...,...,...,...
208,강동경희대병원앞점,서울특별시 강동구 동남로 885 양지빌딩 1층,02-481-9938
209,새문안로점,"서울특별시 종로구 새문안로 91 고려빌딩 1,2층",02-739-5806
210,광화문역1번출구점,서울시 종로구 새문안로5가길 7 1층 일부,02-723-3360
211,성균관대법학관점,서울시 종로구 성균관로 25-2 성균관대학교 법학관 B2,02-740-1958


In [76]:
import requests
import numpy as np

def get_location(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
    headers = {'Authorization':'KakaoAK e80cbc693d356aa131bc7ad0f62699da'}
    res = requests.get(url, headers=headers).json()
    if res['documents']:
        x = res['documents'][0]['x']
        y = res['documents'][0]['y']
        
    else:
        x = np.nan
        y = np.nan
    return x, y

In [77]:
x, y = get_location('서울시 강남구 영동대로 607')
x, y

('127.059486615422', '37.5145473024381')

In [78]:
store = pd.read_csv('커피빈매장.csv')
store['x'] = ''
store['y'] = ''
store.head(2)

,Name,Address,Tel,x,y
0,삼성봉은사거리점,"서울시 강남구 영동대로 607 1,2층",02-3443-5618,,
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,,


In [79]:
for i in store.index:
    address = store.loc[i, 'Address']
    x, y = get_location(address)
    store.loc[i, 'x'] = x
    store.loc[i, 'y'] = y

In [80]:
store.isnull().sum()

Name        0
Address     0
Tel         0
x          19
y          19
dtype: int64

In [81]:
store.dropna(inplace=True)
store.isnull().sum()

Name       0
Address    0
Tel        0
x          0
y          0
dtype: int64

In [82]:
store.head(5)

,Name,Address,Tel,x,y
1,코엑스웨스틴서울점,서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드,02-569-3103,127.057115600619,37.5129534255276
2,삼성오크우드점,서울 강남구 테헤란로87길 46 지하 2층,02-3466-8507,127.058205573267,37.5109823973192
3,삼성봉은사로점,서울시 강남구 봉은사로 628 엘슨빌딩1층,02-538-4523,127.064164448221,37.5146694627537
4,도심공항타워점,서울시 강남구 테헤란로87길 36 공항타워 1층,02-2016-5330,127.058597240146,37.5102336492314
5,트레이드타워점,서울시 강남구 영동대로 511 트레이드타워 지하1층,02-803-9994,127.061023782931,37.510356849269


In [83]:
store.to_csv('커피빈맵.csv', index=False)

In [84]:
store = pd.read_csv('커피빈맵.csv')
store.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     194 non-null    object 
 1   Address  194 non-null    object 
 2   Tel      194 non-null    object 
 3   x        194 non-null    float64
 4   y        194 non-null    float64
dtypes: float64(2), object(3)
memory usage: 7.7+ KB


In [85]:
address = '서울시 마포구'
filt = store['Address'].str.contains(address)
df = store[filt]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, 21 to 53
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Name     5 non-null      object 
 1   Address  5 non-null      object 
 2   Tel      5 non-null      object 
 3   x        5 non-null      float64
 4   y        5 non-null      float64
dtypes: float64(2), object(3)
memory usage: 240.0+ bytes


In [92]:
import folium
import warnings
warnings.filterwarnings(action='ignore')

def get_map(address):
    filt = store['Address'].str.contains(address)
    df = store[filt]
    if len(df.index) == 0:
        print('찾을 수 없습니다.')
    else:
        first = df.head(1)
        x = first['x']
        y = first['y']
        map = folium.Map(location=[y, x], zoom_start=15)
        for idx, row in df.iterrows():
            name = row['Name']
            x = row['x']
            y = row['y']
            folium.Marker(location=[y, x], 
                          popup=folium.Popup(f'[{idx}],{name}', max_width=200), 
                          icon=folium.Icon(icon = 'star')).add_to(map)
        map.save(f'{address}.html')

get_map('상암')